In [8]:
import json
import glob

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit import execute
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from qiskit.quantum_info import random_clifford

import random
from qiskit.circuit.library import HGate, SdgGate
from qiskit.circuit import ClassicalRegister

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from qiskit import QuantumCircuit
from qiskit.circuit.library import U3Gate, CZGate, PhaseGate, CXGate
from mbd_utils import construct_random_clifford, cal_z_exp, calc_imbalance, cal_all_z_exp, construct_mbl_circuit, generate_disorder
from gnn import ExpValCircuitGraphModel

In [9]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(FakeLima()) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [69]:
from collections import defaultdict

num_qubit = 4

def construct_random_circuit(two_q_gate_depth):
    if not two_q_gate_depth:
        d = np.random.randint(1, 6)
        rc = random_circuit(num_qubit, d, 1, measure=True)
    elif 1 <= two_q_gate_depth <= 8:
        rc = random_circuit(num_qubit, two_q_gate_depth*2, 2, measure=True)
        while rc.depth(filter_function=lambda x: x[0].num_qubits == 2) != two_q_gate_depth:
            rc = random_circuit(num_qubit, two_q_gate_depth*2, 2, measure=True)
    elif 9 <= two_q_gate_depth <= 18:
        assert two_q_gate_depth%2 == 0
        half_depth = two_q_gate_depth // 2
        rc1 = random_circuit(num_qubit, two_q_gate_depth, 2, measure=False)
        while rc1.depth(filter_function=lambda x: x[0].num_qubits == 2) != half_depth:
            rc1 = random_circuit(num_qubit, two_q_gate_depth, 2, measure=False)
        rc2 = random_circuit(num_qubit, two_q_gate_depth, 2, measure=False)
        while rc2.depth(filter_function=lambda x: x[0].num_qubits == 2) != half_depth:
            rc2 = random_circuit(num_qubit, two_q_gate_depth, 2, measure=False)
        rc = rc1.compose(rc2)
        rc.measure_all()
    else:
        raise NotImplementedError
    return rc

# depths = [2, 7, 12, 17, 22, 27, 32, 37, 42, 47]
depths = [18]

# circuit = transpile(construct_random_circuit(2), backend=backend_noisy, optimization_level=3)
circuit = construct_random_circuit(6)
circuit.draw(fold=-1)


┌─────┐        ┌───┐           ┌───────────────────┐                               ┌─────────────┐┌──────────────┐┌───┐     ┌───┐      ┌────┐ ┌───┐                                                                     ┌──────────────┐┌─┐      
q_0: ┤ Sdg ├────────┤ H ├───────────┤ U2(6.2597,4.3534) ├────────────■──────────────────┤ Rz(0.57358) ├┤1             ├┤ X ├─────┤ T ├──────┤ √X ├─┤ H ├───────────────────────────────────────────────────■─────────────────┤1             ├┤M├──────
     └┬───┬┘┌───────┴───┴───────┐   └───────┬───┬───────┘            │U1(3.7123)        └┬────────────┤│  Rxx(5.9134) │└─┬─┘┌────┴───┴─────┐└────┘ ├───┤ ┌─────────────────────────┐    ┌────┐             │                 │  Ryy(6.2266) │└╥┘┌─┐   
q_1: ─┤ H ├─┤ U2(0.50585,2.126) ├───────────┤ H ├────────────────────■───────────────────┤ Ry(5.2827) ├┤0             ├──┼──┤0             ├──■────┤ S ├─┤ U(1.3213,3.1563,5.6972) ├────┤ √X ├─────────────┼─────────────────┤0             ├─╫─┤M├───
      └─┬─┘ └───┬────────────┬──┘┌──────────┴───┴───────────┐┌─────────────────────────┐ └───┬───┬────┘└────┬───┬─────┘  │  │  Rzx(6.1859) │  │   ┌┴───┴┐└──────────┬───┬──────────┘┌───┴────┴───┐         │P(1.8295)        └┬───────────┬─┘ ║ └╥┘┌─┐
q_2: ───■───────┤ Ry(3.6818) ├───┤0                         ├┤ U(5.0256,5.5094,4.0293) ├─────┤ S ├──────────┤ Y ├────────┼──┤1             ├──┼───┤ Tdg ├───────────┤ T ├───────────┤ Rz(4.6848) ├─────────■──────────────────┤ U1(4.562) ├───╫──╫─┤M├
      ┌───┐     ├───────────┬┘   │  {XX-YY}(5.1485,0.70605) │└──┬───────────────────┬──┘     └─┬─┘         ┌┴───┴─┐      │  └──────────────┘  │   └┬───┬┘           ├───┤           ├────────────┤┌─────────────────────────┐ └────┬─┬────┘   ║  ║ └╥┘
q_3: ─┤ X ├─────┤ P(1.7824) ├────┤1                         ├───┤ U2(3.6623,5.9928) ├──────────■───────────┤ √Xdg ├──────■────────────────────■────┤ Z ├────────────┤ T ├───────────┤ Rx(1.4528) ├┤ U3(2.7443,6.1476,2.299) ├──────┤M├────────╫──╫──╫─
      └───┘     └───────────┘    └──────────────────────────┘   └───────────────────┘                      └──────┘                                └───┘            └───┘           └────────────┘└─────────────────────────┘      └╥┘        ║  ║  ║ 
c: 4/═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩═════════╩══╩══╩═
                                                                                                                                                                                                                                    3         0  1  2

In [70]:
circuit.depth(lambda x: x[0].num_qubits == 2)

6

In [71]:
qcn = transpile(circuit,backend=backend_noisy)
qcn.depth(lambda x: x[0].num_qubits == 2)

26

In [73]:
qcn.draw()

global phase: 6.258
                 ┌───┐    ┌────────────┐                               »
      q_3 -> 0 ──┤ X ├────┤ Rz(2.6471) ├───────────────────────────────»
                 └───┘    └────────────┘                               »
      q_2 -> 1 ─────────────────────────────────────────────────────■──»
                 ┌────┐  ┌─────────────┐   ┌────┐   ┌────────────┐  │  »
      q_0 -> 2 ──┤ √X ├──┤ Rz(-1.9297) ├───┤ √X ├───┤ Rz(1.5473) ├──┼──»
               ┌─┴────┴─┐└────┬────┬───┘┌──┴────┴──┐└────────────┘┌─┴─┐»
      q_1 -> 3 ┤ Rz(-π) ├─────┤ √X ├────┤ Rz(3π/4) ├──────────────┤ X ├»
               └────────┘     └────┘    └──────────┘              └───┘»
ancilla_0 -> 4 ────────────────────────────────────────────────────────»
                                                                       »
          c: 4/════════════════════════════════════════════════════════»
                                                                       »
«                                                                   »
«      q_3 -> 0 ────────────────────────────────────────────────────»
«                   ┌────┐    ┌───────────┐     ┌────┐    ┌────────┐»
«      q_2 -> 1 ────┤ √X ├────┤ Rz(2.111) ├─────┤ √X ├────┤ Rz(-π) ├»
«                   └────┘    └───────────┘     └────┘    └────────┘»
«      q_0 -> 2 ────────────────────────────────────────────────────»
«               ┌────────────┐    ┌────┐   ┌─────────────┐  ┌────┐  »
«      q_1 -> 3 ┤ Rz(1.9162) ├────┤ √X ├───┤ Rz(-1.3125) ├──┤ √X ├──»
«               └────────────┘    └────┘   └─────────────┘  └────┘  »
«ancilla_0 -> 4 ────────────────────────────────────────────────────»
«                                                                   »
«          c: 4/════════════════════════════════════════════════════»
«                                                                   »
«                   ┌───┐       ┌────┐  ┌─────────────┐     ┌────┐    »
«      q_3 -> 0 ────┤ X ├───────┤ √X ├──┤ Rz(0.56732) ├─────┤ √X ├────»
«                   └─┬─┘     ┌─┴────┴─┐└────┬────┬───┘┌────┴────┴───┐»
«      q_2 -> 1 ──────■───────┤ Rz(-π) ├─────┤ √X ├────┤ Rz(0.56732) ├»
«                             └────────┘     └────┘    └─────────────┘»
«      q_0 -> 2 ──────────────────────────────────────────────────────»
«               ┌────────────┐                                        »
«      q_1 -> 3 ┤ Rz(2.3533) ├────────────────────────────────────────»
«               └────────────┘                                        »
«ancilla_0 -> 4 ──────────────────────────────────────────────────────»
«                                                                     »
«          c: 4/══════════════════════════════════════════════════════»
«                                                                     »
«               ┌────────┐┌───┐┌─────────────┐┌────┐┌──────────────┐      »
«      q_3 -> 0 ┤ Rz(-π) ├┤ X ├┤ Rz(-2.7259) ├┤ √X ├┤ Rz(-0.26467) ├──────»
«               └─┬────┬─┘└─┬─┘└┬────────────┤├────┤├─────────────┬┘┌────┐»
«      q_2 -> 1 ──┤ √X ├────■───┤ Rz(1.9663) ├┤ √X ├┤ Rz(-2.2149) ├─┤ √X ├»
«                 └────┘        └────────────┘└────┘└─────────────┘ └────┘»
«      q_0 -> 2 ──────────────────────────────────────────────────────────»
«                                                                         »
«      q_1 -> 3 ──────────────────────────────────────────────────────────»
«                                                                         »
«ancilla_0 -> 4 ──────────────────────────────────────────────────────────»
«                                                                         »
«          c: 4/══════════════════════════════════════════════════════════»
«                                                                         »
«                                                    ┌────────┐   ┌────┐   »
«      q_3 -> 0 ─────────────────■────────────────■──┤ Rz(-π) ├───┤ √X ├───»
«               ┌─────────────┐┌─┴─┐┌──────

In [63]:
from qiskit import QuantumRegister, ClassicalRegister
qr = QuantumRegister(2)
qc = QuantumCircuit(qr)
qc.cz(0, 1)
qc.depth(lambda x: x[0].num_qubits == 2)

1

In [7]:
generate_data(construct_random_circuit, depths, './data/haoran_mbl/rc_train', 500)

  0%|          | 0/500 [00:00<?, ?it/s]

In [24]:
f = open('./data/haoran_mbl/rc_train/random_circuits_depth4.json')
out = json.load(f)

In [25]:
out[1]['circuit_depth']

36